In [1]:
import torch.nn as nn

In [2]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        #A
        self.register_buffer('mask',torch.triu(torch.ones(context_length, context_length),diagonal=1))
    #B
    def forward(self, x):
        b, num_tokens, d_in = x.shape
    #C
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.transpose(1, 2)#C
        attn_scores.masked_fill_(#D
        self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        context_vec = attn_weights @ values
        return context_vec

In [7]:
import torch
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your(x^1)
    [0.55, 0.87, 0.66], # journey(x^2)
    [0.57, 0.85, 0.64], # starts(x^3)
    [0.22, 0.58, 0.33], # with(x^4)
    [0.77, 0.25, 0.10], # one(x^5)
    [0.05, 0.80, 0.55]] # step(x^6)
)
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

x_2 = inputs[1]#A
d_in = inputs.shape[1]#B
d_out = 2#C

torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)
print("context_vecs.shape:", context_vecs.shape)

torch.Size([2, 6, 3])
context_vecs.shape: torch.Size([2, 6, 2])


## 3.6.1 Stacking multiple single-head attention layers

In [8]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList([CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) for _ in range(num_heads)])
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [11]:
torch.manual_seed(123)
context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


In [12]:
class MultiHeadAttention(nn.Module):
	def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
		super().__init__()
		assert d_out % num_heads == 0, "d_out must be divisible by num_heads"
		self.d_out = d_out
		self.num_heads = num_heads
		self.head_dim = d_out // num_heads
		#A
		self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
		self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
		self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
		self.out_proj = nn.Linear(d_out, d_out)
		#B
		self.dropout = nn.Dropout(dropout)
		self.register_buffer('mask',torch.triu(torch.ones(context_length, context_length), diagonal=1))
	def forward(self, x):
		b, num_tokens, d_in = x.shape
		keys = self.W_key(x)#C
		queries = self.W_query(x)#C
		values = self.W_value(x)#C
		keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) #D
		values = values.view(b, num_tokens, self.num_heads, self.head_dim) #D
		queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)#D
		keys = keys.transpose(1, 2)#E
		queries = queries.transpose(1, 2)#E
		values = values.transpose(1, 2)#E
		attn_scores = queries @ keys.transpose(2, 3)
		#F
		mask_bool = self.mask.bool()[:num_tokens, :num_tokens]#G
		attn_scores.masked_fill_(mask_bool, -torch.inf)#H
		attn_weights = torch.softmax(
		attn_scores / keys.shape[-1]**0.5, dim=-1)
		attn_weights = self.dropout(attn_weights)
		context_vec = (attn_weights @ values).transpose(1, 2) #I
		#J
		context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
		context_vec = self.out_proj(context_vec)
		#K
		return context_vec


#A Reduce the projection dim to match desired output dim
#B Use a Linear layer to combine head outputs
#C Tensor shape: (b, num_tokens, d_out)
#D We implicitly split the matrix by adding a `num_heads` dimension. Then we unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
#E Transpose from shape (b, num_tokens, num_heads, head_dim) to (b, num_heads, num_tokens, head_dim)
#F Compute dot product for each head
#G Mask truncated to the number of tokens
#H Use the mask to fill attention scores
#I Tensor shape: (b, num_tokens, n_heads, head_dim)
#J Combine heads, where self.d_out = self.num_heads * self.head_dim
#K Add an optional linear projection

In [14]:
torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])
